In [1]:
import pandas as pd
import numpy as np
import re
from gensim.models import word2vec, Word2Vec
import matplotlib.pyplot as plt
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation
import numpy as np
from numpy import argmax
%matplotlib inline

from ckonlpy.tag import Twitter #konlpy twitter는 add_dictionary가 없어서, ckonlpy(customized konlpy) 사용.
twitter = Twitter()

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


## 입력데이터

In [2]:
df = pd.read_csv('final_data_19613.csv', sep='\t', encoding='UTF-8')
# df = df2.loc[df2['대상 선정']=='O',:]

In [3]:
df = df.fillna('N')

userdic = pd.read_csv('userdic_1906122.csv', sep='\t', encoding='UTF-8')
userdic['User Dic.'][0:1]
userdic = userdic.rename(columns = {'User Dic.':'standard'})
userdic = userdic.rename(columns = {'동의어':'missprint'})
userdic = userdic.rename(columns = {'불용단어':'stops'})
userdic_list = userdic.rename(columns = {'User Dic.':'standard'})
userdic_list = userdic.rename(columns = {'동의어':'missprint'})
userdic_list = userdic.rename(columns = {'불용단어':'stops'})
userdic_list = userdic_list.fillna('')
userdic = userdic.fillna('')
replace_word = userdic.iloc[:,1:3]
replace_word = replace_word[replace_word.missprint != '']

In [4]:
replaceword_list =  replace_word['standard'].tolist()
missword_list = replace_word['missprint'].tolist()
print(len(replaceword_list))
print(len(missword_list))

757
757


In [5]:
for i in range(len(replaceword_list)):
    df['clm_prtl_sbc'] = df['clm_prtl_sbc'].replace(missword_list[i],replaceword_list[i],regex=True)
#regex=True 설정을 주면 조사가 붙어있던 말던 단어를 다 바꿔준다.

## 텍스트 전처리

In [6]:
### (1) 오타, 동의어 처리(replace)
### (2) 사용자 단어 정의(User dictionary)

for word in replaceword_list: 
    twitter.add_dictionary(word, 'Noun') # for문을 통해 dictionary에 단어들을 Noun으로 추가.

# twitter.pos(df['clm_prtl_sbc'][1]) # '리어', '미장착'이 Noun으로 잘 정의되었는지 확인.

### (3) 명사 추출(Twitter)

def nouns(text):
    text = str(text)
    text = twitter.nouns(text)
    return text

df['clm_prtl_sbc'] = df['clm_prtl_sbc'].apply(nouns)

### (4) 특수문자, 띄어쓰기 전처리

def preprocessing(text):
    # 개행문자 제거
    text= str(text)
    text = re.sub('\\\\n', ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    #text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ]', ' ', text) 
    text = re.sub(' +', ' ', text)
    return text

# 불용어 제거
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = ['(LH)','(RH)','(사진첨부)','(우)','(좌)','[C]','A/S','C','C-10000','C-10000','F/LH','F/RH','FR','FRT',
             'Frt','frt','L/H','L/H리어','LH','LH/RH','LH쪽','R/H','R/H리어','R/LH','R/RH','RH','RH쪽','Rr','RR','rr',
             '결과','고객','고객에게','공임','광주','그랜저','그랜져','낫','낮','넥쏘','니로','당','당차량','당차량은','대리점',
             '동영상참조','레이','렌트','리무진','리어LH','리어RH','모닝','모하비','밤','벨로스터','사진','사진참조','사진첨부',
             '상기','상기차량은','새벽','새차','서비스','소하','스타렉스','스팅어','스포티지','신차','신차','심야','싼타페',
             '써비스','쏘나타','쏘렌토','쏘울','아반떼','아산','아이오닉','아침','야간','언덕','엑센트','열','울산','원인',
             '임시 넘버','임시 번호','입고','자동차','저머검내용','점검/원인','점검결과','점검내용','점검및원인','점검및조치내용',
             '점검점검','점검점검내용','조치결과','조치내용','주간','주재원','주제원','주차장','증상','지하','진입','차량',
             '차량은','철도','첨부','첨부파일','청구','출고','카 마스터','카니발','카렌스','코나','클레임','투싼',
             '팰리세이드','하이리무진','하이브리드','현     상','현    상','현   상','현  상','현 상','현산','현상','현상및점검',
             '현상및조치','화성'
]
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words) #stops에 안 걸리는 나머지 것들 이어붙이기

%time df['clm_prtl_sbc'] = df['clm_prtl_sbc'].apply(preprocessing)
%time df['clm_prtl_sbc'] = df['clm_prtl_sbc'].apply(remove_stopwords)
df['clm_prtl_sbc'].head()

Wall time: 819 ms
Wall time: 691 ms


0     엔진 룸 다량 연료 누유 발생 점검 엔진 룸 다량 연료 누유 점검 점검 내용 엔진...
1                                리어 도어 안티 치핑 필름 장착 불량 
2                      커튼 홀더 장착 불량 조치 내용 커튼 홀더 장착 후 함 
3                  및 조치 시 웨더 스트림 스트림 무 주름 발생 들뜸 교환 조치 
4                             몰딩 내부 고무 조립 불량 상태 몰딩 조립 
Name: clm_prtl_sbc, dtype: object

In [7]:
category_count = df['대상 선정'].value_counts()
category_count

X    6298
O    4331
Name: 대상 선정, dtype: int64

In [8]:
category_count = df['구분'].value_counts()
category_count

N          6298
기능(커넥터)    2244
실내부품이음     1589
하체이음        498
Name: 구분, dtype: int64

In [9]:
category_count = df['책임구'].value_counts()
category_count

N       6298
구매      3306
생산공장    1012
연구소       13
Name: 책임구, dtype: int64

In [10]:
category_count = df['원인'].value_counts()
category_count

N        6298
부품       3306
조립불량      301
결선불량      283
기타        203
체결불량      135
미결선        50
이종/간섭      30
설계/구조      13
이물         10
Name: 원인, dtype: int64

In [11]:
category_count = df['검토결과'].value_counts()
category_count

X---                    6298
O-기능(커넥터)-구매-부품         1801
O-실내부품이음-구매-부품          1165
O-하체이음-구매-부품             340
O-기능(커넥터)-생산공장-결선불량      283
O-실내부품이음-생산공장-조립불량       232
O-실내부품이음-생산공장-체결불량        91
O-기능(커넥터)-생산공장-기타         80
O-실내부품이음-생산공장-기타          78
O-하체이음-생산공장-조립불량          69
O-기능(커넥터)-생산공장-미결선        50
O-하체이음-생산공장-기타            45
O-하체이음-생산공장-체결불량          44
O-기능(커넥터)-생산공장-이종/간섭      30
O-실내부품이음-연구소-설계/구조        13
O-실내부품이음-생산공장-이물          10
Name: 검토결과, dtype: int64

In [12]:
# 예측값과 실제값 비교를 위해 컬럼을 하나 더 생성합니다.
# df['result_pred'] = df['검토결과'].astype(str).copy()
df['target_pred'] = df['대상 선정'].astype(str).copy()
# df['charge_pred'] = df['책임구'].astype(str).copy()
# df['division_pred'] = df['구분'].astype(str).copy()

## X, y 정의

In [13]:
from sklearn.preprocessing import LabelEncoder
y_encod = df['대상 선정']
classle = LabelEncoder()
y_encod = classle.fit_transform(df['대상 선정'].values) # y값 인코딩.
print(np.unique(y_encod))

[0 1]


In [14]:
# ro column을 list형태로 ro_text 변수에 할당.
ro_text = list(df['clm_prtl_sbc'])
sentiments = list(y_encod) # y값.

In [15]:
ro_text[0]

' 엔진 룸 다량 연료 누유 발생 점검 엔진 룸 다량 연료 누유 점검 점검 내용 엔진 룸 프런트 저압 라인 연료 공급 파이프 중 고압 펌프 연결 저압 라인 퀵 커넥터 터 오 체결 랭킹 중 다량 료 엔진 룸 내부 고압 연료 펌프 하단 바닥 누유 발생 함 고압 펌프 저압 라인 퀵 커넥터 터 오 체결 연료 펌프 구동 중 이탈 다량 연료 엔진 룸및 다닥 누유 사항 임 조치 내용 고압 펌측 퀵 턴텍터 체결 및 엔진 룸 가솔린 오염 부위 세척 시동 후 누유 확인 테스트 요망 사항 해당 저압 파이프 이탈 시 파이프 하단 부위 익 죠스 트 매니폴드 이 어서 가솔린 연료 발화 인하 회재 발생 수 있음 중요 부위 조립 철저 관리 필요 사항 임 '

In [16]:
sentiments[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [17]:
sentences = []
for text in ro_text:
    sentences.append(text.split())

In [18]:
sentences[:3]

[['엔진',
  '룸',
  '다량',
  '연료',
  '누유',
  '발생',
  '점검',
  '엔진',
  '룸',
  '다량',
  '연료',
  '누유',
  '점검',
  '점검',
  '내용',
  '엔진',
  '룸',
  '프런트',
  '저압',
  '라인',
  '연료',
  '공급',
  '파이프',
  '중',
  '고압',
  '펌프',
  '연결',
  '저압',
  '라인',
  '퀵',
  '커넥터',
  '터',
  '오',
  '체결',
  '랭킹',
  '중',
  '다량',
  '료',
  '엔진',
  '룸',
  '내부',
  '고압',
  '연료',
  '펌프',
  '하단',
  '바닥',
  '누유',
  '발생',
  '함',
  '고압',
  '펌프',
  '저압',
  '라인',
  '퀵',
  '커넥터',
  '터',
  '오',
  '체결',
  '연료',
  '펌프',
  '구동',
  '중',
  '이탈',
  '다량',
  '연료',
  '엔진',
  '룸및',
  '다닥',
  '누유',
  '사항',
  '임',
  '조치',
  '내용',
  '고압',
  '펌측',
  '퀵',
  '턴텍터',
  '체결',
  '및',
  '엔진',
  '룸',
  '가솔린',
  '오염',
  '부위',
  '세척',
  '시동',
  '후',
  '누유',
  '확인',
  '테스트',
  '요망',
  '사항',
  '해당',
  '저압',
  '파이프',
  '이탈',
  '시',
  '파이프',
  '하단',
  '부위',
  '익',
  '죠스',
  '트',
  '매니폴드',
  '이',
  '어서',
  '가솔린',
  '연료',
  '발화',
  '인하',
  '회재',
  '발생',
  '수',
  '있음',
  '중요',
  '부위',
  '조립',
  '철저',
  '관리',
  '필요',
  '사항',
  '임'],
 ['리어', '도어', '안티', '치핑', '필름', '장착',

## Word2Vec

In [19]:
# Word2Vec의 파라미터 값들 정의.
num_features = 300    
min_word_count = 40   
num_workers = 4       
context = 10          
downsampling = 1e-3 

In [20]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
   level=logging.INFO)

In [21]:
# word2vec으로 단어 학습.
from gensim.models import word2vec

model = word2vec.Word2Vec(sentences, workers=num_workers, \
           size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

2019-06-13 14:47:49,132 : INFO : collecting all words and their counts
2019-06-13 14:47:49,133 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-06-13 14:47:49,195 : INFO : PROGRESS: at sentence #10000, processed 270726 words, keeping 6823 word types
2019-06-13 14:47:49,200 : INFO : collected 7041 word types from a corpus of 288386 raw words and 10629 sentences
2019-06-13 14:47:49,202 : INFO : Loading a fresh vocabulary
2019-06-13 14:47:49,214 : INFO : effective_min_count=40 retains 714 unique words (10% of original 7041, drops 6327)
2019-06-13 14:47:49,218 : INFO : effective_min_count=40 leaves 258592 word corpus (89% of original 288386, drops 29794)
2019-06-13 14:47:49,229 : INFO : deleting the raw counts dictionary of 7041 items
2019-06-13 14:47:49,232 : INFO : sample=0.001 downsamples 66 most-common words
2019-06-13 14:47:49,233 : INFO : downsampling leaves estimated 163258 word corpus (63.1% of prior 258592)
2019-06-13 14:47:49,238 : INFO : estimated 

In [22]:
# Word2Vec의 feature vector를 얻는 함수.
def get_features(words, model, num_features):
    feature_vector = np.zeros((num_features),dtype=np.float32)

    num_words = 0
    index2word_set = set(model.wv.index2word)

    for w in words:
        if w in index2word_set:
            num_words += 1
            feature_vector = np.add(feature_vector, model[w])

    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

In [23]:
def get_dataset(ro_text, model, num_features):
    dataset = list()

    for s in ro_text:
        dataset.append(get_features(s, model, num_features))

    reviewFeatureVecs = np.stack(dataset)
    
    return reviewFeatureVecs

In [24]:
test_data_vecs = get_dataset(sentences, model, num_features)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


In [25]:
test_data_vecs

array([[-0.18820891,  0.3740204 , -0.02320145, ...,  0.12897149,
        -0.13520084, -0.05709308],
       [ 0.25823545, -0.66226953,  0.16248675, ...,  0.09090146,
        -0.07563614, -0.05514698],
       [ 0.16198148, -0.60536104,  0.2981511 , ...,  0.14000596,
        -0.2065518 , -0.16763677],
       ...,
       [-0.03858716,  0.20697872,  0.03257219, ...,  0.19422221,
        -0.08036138,  0.12046459],
       [-0.2997496 ,  0.67396855, -0.05933732, ...,  0.20588255,
         0.08431955,  0.10612082],
       [-0.12377922,  0.4750918 , -0.00717465, ...,  0.22378506,
        -0.01475985,  0.00290286]], dtype=float32)

In [26]:
len(test_data_vecs)

10629

# 1. RNN

### Train, Test data Split - 1안 (RNN)

In [ ]:
X = test_data_vecs
y = np.array(sentiments)

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
print(X)
print(y)

In [ ]:
maxlen = 138
training_samples = 3898
test_samples = 10000

In [ ]:
indices = np.arange(X.shape[0])
indices2 = np.arange(y.shape[0])
np.random.shuffle(indices)
np.random.shuffle(indices2)
data_x = X[indices]
data_y = y[indices2]

In [ ]:
x_train = data_x[:training_samples]
y_train = data_y[:training_samples]
x_test = data_x[training_samples: training_samples + test_samples]
y_test = data_y[training_samples: training_samples + test_samples]

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
print(x_train)
print(y_train)
print(x_test)
print(y_test)

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding,Dense,Flatten,SimpleRNN

max_features = 10000
# maxlen = 163
# embedding_dim = 163

model = Sequential()
model.add(Embedding(max_features,32))
model.add(SimpleRNN(32))
model.add(Dense(1,activation='sigmoid'))

# model.add(Embedding(max_features,embedding_dim,input_length=maxlen))
# model.add(Flatten())
# model.add(Dense(32, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
history = model.fit(x_train,y_train,epochs=10,batch_size=128,validation_data=(x_test,y_test))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
print(history.history['acc'])
print(history.history['val_acc'])
print(history.history['loss'])
print(history.history['val_loss'])

In [ ]:
print(round(np.mean(history.history['acc'])*100,2))
print(round(np.mean(history.history['val_acc'])*100,2))

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

In [ ]:
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.figure()

In [ ]:
# model.load_weights('pre_trained_model.h5')
model.evaluate(x_test, y_test)

# 2. Random Forest

### Train, Test data Split - 2안 (RandomForest)

In [27]:
x = test_data_vecs
y = np.array(sentiments)

In [28]:
x.shape

(10629, 300)

In [29]:
y.shape

(10629,)

In [31]:
print(x)
print(y)

[[-0.18820891  0.3740204  -0.02320145 ...  0.12897149 -0.13520084
  -0.05709308]
 [ 0.25823545 -0.66226953  0.16248675 ...  0.09090146 -0.07563614
  -0.05514698]
 [ 0.16198148 -0.60536104  0.2981511  ...  0.14000596 -0.2065518
  -0.16763677]
 ...
 [-0.03858716  0.20697872  0.03257219 ...  0.19422221 -0.08036138
   0.12046459]
 [-0.2997496   0.67396855 -0.05933732 ...  0.20588255  0.08431955
   0.10612082]
 [-0.12377922  0.4750918  -0.00717465 ...  0.22378506 -0.01475985
   0.00290286]]
[1 1 1 ... 1 0 1]


In [35]:
train_samples = 8530
test_samples = 10000

x_train = x[:train_samples]
x_test = x[train_samples:train_samples + test_samples]
y_train = y[:train_samples]
y_test = y[train_samples:train_samples + test_samples]

In [36]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(8530, 300)
(2099, 300)
(8530,)
(2099,)


In [67]:
np.any(np.isnan(x_train))

True

In [79]:
print(x_train[:,0].shape)
index = 0
for i in x_train[:,0]:
    if not np.isfinite(i):
        print(index, i)
    index +=1

(8530,)


In [80]:
x_train = np.nan_to_num(x_train)
x_train

array([[-0.18820891,  0.3740204 , -0.02320145, ...,  0.12897149,
        -0.13520084, -0.05709308],
       [ 0.25823545, -0.66226953,  0.16248675, ...,  0.09090146,
        -0.07563614, -0.05514698],
       [ 0.16198148, -0.60536104,  0.2981511 , ...,  0.14000596,
        -0.2065518 , -0.16763677],
       ...,
       [-0.20766437,  0.5487742 , -0.06931449, ...,  0.22313634,
         0.04732496,  0.07341839],
       [-0.20193511,  0.9208285 ,  0.15320879, ...,  0.37161928,
        -0.22806491,  0.05464142],
       [-0.04460226,  0.09273315, -0.08345181, ...,  0.00733201,
        -0.11814306, -0.05732635]], dtype=float32)

In [68]:
np.any(np.isnan(y_train))

False

In [69]:
np.any(np.isnan(x_test))

True

In [83]:
print(x_test[:,0].shape)
index = 0
for i in x_test[:,0]:
    if not np.isfinite(i):
        print(index, i)
    index +=1

(2099,)


In [82]:
x_test = np.nan_to_num(x_test)
x_test

array([[ 0.10683598, -0.48255116,  0.11756403, ...,  0.08395084,
        -0.05631314, -0.04076826],
       [-0.1036638 , -0.38513365,  0.04924913, ...,  0.05874881,
         0.06136845, -0.00979206],
       [ 0.06354552, -0.22544909, -0.10788536, ...,  0.03393255,
        -0.06591561, -0.02888705],
       ...,
       [-0.03858716,  0.20697872,  0.03257219, ...,  0.19422221,
        -0.08036138,  0.12046459],
       [-0.2997496 ,  0.67396855, -0.05933732, ...,  0.20588255,
         0.08431955,  0.10612082],
       [-0.12377922,  0.4750918 , -0.00717465, ...,  0.22378506,
        -0.01475985,  0.00290286]], dtype=float32)

In [84]:
np.any(np.isnan(y_test))

False

## RandomForestClassifier로 학습

In [85]:
from sklearn.ensemble import RandomForestClassifier

# 랜덤포레스트 분류기를 사용
forest = RandomForestClassifier(n_estimators = 100, n_jobs = -1, random_state=42)
forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [86]:
forest = forest.fit(x_train, y_train)

## 모델평가

In [88]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=5, shuffle=True, random_state=0)

scoring = 'accuracy'
%time score = cross_val_score(forest, x_train, y_train, cv=k_fold, n_jobs=-1, scoring=scoring)
score

Wall time: 35.7 s


array([0.82063306, 0.80363423, 0.81242673, 0.82649472, 0.82766706])

In [89]:
round(np.mean(score)*100,2)

81.82

## 예측

In [90]:
y_predict = forest.predict(x_test)
y_predict[:3]

array([1, 1, 0])

In [91]:
y_predict.shape

(2099,)

In [93]:
output = pd.DataFrame (data = {'test_target_pred':y_predict})
output.head()

test_target_pred
0                 1
1                 1
2                 0
3                 1
4                 1

In [98]:
y_predict

array([1, 1, 0, ..., 1, 1, 1])

In [97]:
df['test_target_pred'] = y_predict

ValueError: Length of values does not match length of index

In [96]:
df.head(5)

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd    ceq_psit_nm  \
0               W  2017     20170811         ZZ7           N  조립 문제_미장착/오장착   
1               W  2018     20180822         ZZ7         B11  조립 문제_미장착/오장착   
2               W  2018     20181006         ZZ5         B11  조립 문제_미장착/오장착   
3               W  2018     20180928         ZZ7         B11  조립 문제_미장착/오장착   
4               W  2016     20160526         ZZ5         B11  조립 문제_미장착/오장착   

  trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
0         KV11          CK     J5          TT        82305   
1         KV11          YP     A9          TT        84211   
2         KV11          YP     A9          TT        83912   
3         KV11          YP     A9          TT        83120   
4         KV11          YP     A9          TT        83860   

                                        clm_prtl_sbc 대상 선정 구분 책임구 원인  검토결과  \
0   엔진 룸 다량 연료 누유 발생 점검 엔진 룸 다량 연료 누유 점검 점검 내용 엔진...     X  N   N  N  X---   
1                              리어 도어 안티 치핑 필름 장착 불량      X  N   N  N  X---   
2                    커튼 홀더 장착 불량 조치 내용 커튼 홀더 장착 후 함      X  N   N  N  X---   
3                및 조치 시 웨더 스트림 스트림 무 주름 발생 들뜸 교환 조치      X  N   N  N  X---   
4                           몰딩 내부 고무 조립 불량 상태 몰딩 조립      X  N   N  N  X---   

  target_pred  
0           X  
1           X  
2           X  
3           X  
4           X

In [ ]:
df_test ['predict_diff'] = 0
df_test ['predict_diff'] = (df_test['검토결과'] == df_test['test_result_pred']) == 1
df_test['predict_diff'] = df_test['predict_diff'].astype(int)
df_test.head(3)

In [ ]:
diff_sum = df_test['predict_diff'].sum()
diff_sum

In [ ]:
y_predict_len = len(y_predict)
y_predict_len

In [100]:
from sklearn.metrics import accuracy_score
# 테스트 데이터를 넣고 예측합니다.
y_pred = forest.predict(x_test)
y_pred[:3]

array([1, 1, 0])

In [101]:
print(accuracy_score(y_test, y_pred))

0.808480228680324
